In [ ]:
import pandas as pd

In [ ]:
s1 = pd.read_csv('/kaggle/input/datasets/kagglertw/pred-f5/sub1.csv')
s2 = pd.read_csv('/kaggle/input/datasets/kagglertw/pred-f5/sub2.csv')
s3 = pd.read_csv('/kaggle/input/datasets/kagglertw/pred-f5/sub3.csv')
s4 = pd.read_csv('/kaggle/input/datasets/kagglertw/pred-f5/sub4.csv')
s5 = pd.read_csv('/kaggle/input/datasets/kagglertw/pred-f5/sub5.csv')

In [ ]:
final = pd.read_csv('/kaggle/input/notebooks/kagglertw/final-filter/submission.csv')

In [ ]:
with open("/kaggle/input/notebooks/kagglertw/shemagh-simple-ddq-map-50-o-inference6/problematic_cases.txt", "r") as f:
    problematic_cases = [line.strip() for line in f]

In [ ]:
problematics = problematic_cases

In [ ]:
dic = {}

for i in problematics:
    print(i)

    p  = final[final['filename'] == i]['prediction_string'].unique()
    s1_ = s1[s1['filename'] == i]['prediction_string'].unique()
    s2_ = s2[s2['filename'] == i]['prediction_string'].unique()
    s3_ = s3[s3['filename'] == i]['prediction_string'].unique()
    s4_ = s4[s4['filename'] == i]['prediction_string'].unique()
    s5_ = s5[s5['filename'] == i]['prediction_string'].unique()

    print(s1_)
    print(s2_)
    print(s3_)
    print(s4_)
    print(s5_)

    dic[i] = (p, s1_, s2_, s3_, s4_, s5_)

    print('


In [ ]:
def parse_prediction_string(pred_str):
    if pred_str == '-' or pred_str.strip() == '':
        return []
    nums = pred_str.strip().split()
    boxes = []
    for i in range(0, len(nums), 6):
        if i + 6 > len(nums):
            break
        cls = int(nums[i])
        conf = float(nums[i+1])
        box = nums[i:i+6]
        boxes.append((cls, conf, box))
    return boxes


final_dict = {}

for img_id, preds_tuple in dic.items():

    
    all_preds = []
    for arr in preds_tuple:
        for p in arr:
            if p != '-' and p.strip() != '':
                all_preds.append(p.strip())

    if not all_preds:
        final_dict[img_id] = '-'
        continue

    
    all_parsed = [parse_prediction_string(p) for p in all_preds]

    
    all_boxes = [box for boxes in all_parsed for box in boxes]

    
    all_single = all(
        sum(1 for b in boxes if b[0] == 0) <= 1 and
        sum(1 for b in boxes if b[0] == 1) <= 1
        for boxes in all_parsed
    )

    
    
    
    
    if all_single:
        class_0_boxes = [b for b in all_boxes if b[0] == 0]
        class_1_boxes = [b for b in all_boxes if b[0] == 1]

        tokens = []
        if class_0_boxes:
            best_0 = max(class_0_boxes, key=lambda x: x[1])
            tokens += best_0[2]
        if class_1_boxes:
            best_1 = max(class_1_boxes, key=lambda x: x[1])
            tokens += best_1[2]

        final_dict[img_id] = " ".join(tokens)

    
    
    
    
    
    else:
        search_pool = all_preds[1:] if len(all_preds) > 1 else all_preds
        best_str = "-"
        max_box_count = -1
        best_avg_conf = -1.0
        for pred in search_pool:
            boxes = parse_prediction_string(pred)
            n = len(boxes)
            avg_conf = sum(b[1] for b in boxes) / n if n > 0 else 0.0
            if n > max_box_count or (n == max_box_count and avg_conf > best_avg_conf):
                max_box_count = n
                best_avg_conf = avg_conf
                best_str = pred
        final_dict[img_id] = best_str



for img_id, result in final_dict.items():
    boxes = parse_prediction_string(result)
    classes = [b[0] for b in boxes]

In [ ]:
final_dict

In [ ]:
for name in final_dict:
    final.loc[final['filename'] == name, 'prediction_string'] = final_dict[name]

In [ ]:

missing_ids = []

for filename in final['filename'].unique():
    
    p_preds = final.loc[final['filename'] == filename, 'prediction_string'].values[0]
    p_classes = []
    if p_preds != '-' and p_preds.strip():
        tokens = p_preds.split()
        
        i = 0
        while i < len(tokens):
            p_classes.append(tokens[i])
            i += 2  
            while i < len(tokens) and tokens[i] not in ['0', '1']:
                i += 1

    
    s_tokens = []
    for s in [s1, s2, s3, s4, s5]:
        s_preds = s[s['filename'] == filename]['prediction_string'].values
        if len(s_preds) == 0:
            continue
        if s_preds[0] != '-':
            s_tokens.extend(s_preds[0].split())

    
    i = 0
    while i < len(s_tokens):
        cls = s_tokens[i]
        conf = float(s_tokens[i + 1])
        if cls in ['0', '1'] and conf > 0.3:
            if cls not in p_classes:
                missing_ids.append(filename)
                break  
        i += 2
        while i < len(s_tokens) and s_tokens[i] not in ['0', '1']:
            i += 1


In [ ]:
for filename in missing_ids:
    
    p_preds = final.loc[final['filename'] == filename, 'prediction_string'].values[0]
    old_preds = p_preds
    p_classes = []
    if p_preds != '-' and p_preds.strip():
        tokens = p_preds.split()
        
        i = 0
        while i < len(tokens):
            cls = tokens[i]
            conf = float(tokens[i+1])
            p_classes.append(cls)
            i += 2  
            

    
    s_tokens = []
    for s in [s1, s2, s3, s4, s5]:
        s_preds = s[s['filename'] == filename]['prediction_string'].values
        if len(s_preds) == 0:
            continue
        if s_preds[0] != '-':
            s_tokens.extend(s_preds[0].split())

    
    i = 0
    missing_preds = []
    while i < len(s_tokens):
        cls = s_tokens[i]
        conf = float(s_tokens[i+1])
        if cls in ['0', '1'] and conf > 0.5 and cls not in p_classes:
            
            j = i + 2
            pred_tokens = [cls, str(conf)]
            while j < len(s_tokens) and s_tokens[j] not in ['0', '1']:
                pred_tokens.append(s_tokens[j])
                j += 1
            missing_preds.append(' '.join(pred_tokens))
            p_classes.append(cls)  
            i = j
        else:
            
            i += 2
            while i < len(s_tokens) and s_tokens[i] not in ['0', '1']:
                i += 1

    
    if missing_preds:
        updated_preds = ('' if p_preds == '-' else p_preds + ' ') + ' '.join(missing_preds)
        final.loc[final['filename'] == filename, 'prediction_string'] = updated_preds

In [ ]:
final.to_csv('final.csv' , index = False)